# Observability

## Forecast first

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import torch
from torch.nn import MSELoss
from torch.optim import Adam

from sklearn.preprocessing import StandardScaler

from distorch.dataset import ForecastingDataset
from distorch.trainer import Trainer
from distorch.metrics import PerformanceEvaluator

from distorch.models.lstm_forecaster import LSTMForecaster
from distorch.models.lstm_disaggregator import LSTMDisaggregator

### load the model

In [ ]:
forecaster = LSTMForecaster()
trainer = Trainer(
    forecaster, MSELoss(reduction='mean'), Adam(forecaster.parameters(), lr=0.001)
)
trainer.load_checkpoint('saved_models/forecaster')
trainer.model.state_dict()

### load the data

Forecasting three days in September

In [ ]:
df = pd.read_csv('datasets/data/forecasting_preprocset.csv', index_col=0, parse_dates=True)

mask = (df.index.month == 9) & (df.index.day.isin([4,5,6]))
X = df[mask].drop(columns=['netload'])
y = df[mask]['netload']

scaler = StandardScaler()
X_s = scaler.fit_transform(X)

X_tensor = torch.as_tensor(X_s).float()
netload_tensor = torch.as_tensor(y).float()
print(X_tensor.shape, y_tensor.shape)

### Forecast

In [ ]:
netload_pred = forecaster.predict(X_tensor)

pd.DataFrame(
    index = df[mask].index
    columns = {
        'netload_pred': netload_pred,
        'netload_true': netload_tensor.numpy()
    }
).plot()

## Disaggregation

### load model

In [ ]:
disaggregator = LSTMDisaggregator()
trainer = Trainer(
    disaggregator, MSELoss(reduction='mean'), Adam(disaggregator.parameters(), lr=0.001)
)
trainer.load_checkpoint('saved_models/disaggregator')
trainer.model.state_dict()

### disaggregation data

In [ ]:
df_dis = pd.read_csv('datasets/data/disaggregation_preprocset.csv', index_col=0, parse_dates=True)

df_dism = df_dis[mask]

X = df_dism.drop(columns=['load', 'pv'])
#switching to forecasted values
X.netload = netload_pred
y = df_dism[['load', 'pv']].values

scaler = StandardScaler()
X_s = scaler.fit_transform(X)

X_tensor = torch.as_tensor(X_s).float()
y_tensor = torch.as_tensor(y).float()
print(X_tensor.shape, y_tensor.shape)

In [ ]:
pv_pred, load_pred = disaggregator.predict(X_tensor)

pd.DataFrame(
    index = df_dism.index
    columns = {
        'pv_pred': pv_pred,
        'pv_true': y_tensor[:,1].numpy()
    }
).plot()

In [ ]:
pd.DataFrame(
    index = df_dism.index
    columns = {
        'load_pred': load_pred,
        'load_true': y_tensor[:,0].numpy()
    }
).plot()

In [ ]:
from evaluation.metrics import PerformanceEvaluator

netload_eval = PerformanceEvaluator(netload_pred, netload_tensor.numpy(), scale=)
print(netload_eval.get_performance_metrics())
pv_eval = PerformanceEvaluator(netload_pred, pv_tensor[:,1].numpy(), scale=1.41)
print(pv_eval.get_performance_metrics())
laod_eval = PerformanceEvaluator(pv_pred, load_tensor[:,0].numpy(), scale=5.18)
print(netload_eval.get_performance_metrics())

## Disaggregation first

In [ ]:
disaggregator = LSTMDisaggregator()
trainer = Trainer(
    disaggregator, MSELoss(reduction='mean'), Adam(disaggregator.parameters(), lr=0.001)
)
trainer.load_checkpoint('saved_models/disaggregator')
trainer.model.state_dict()

In [ ]:
df_dis = pd.read_csv('datasets/data/disaggregation_preprocset.csv', index_col=0, parse_dates=True)

df_dism = df_dis[mask]

X = df_dism.drop(columns=['load', 'pv'])
y = df_dism[['load', 'pv']].values

scaler = StandardScaler()
X_s = scaler.fit_transform(X)

X_tensor = torch.as_tensor(X_s).float()
y_tensor = torch.as_tensor(y).float()
print(X_tensor.shape, y_tensor.shape)